# Quota Subscription with a Feature Workflow

Welcome to the workflow demo for creating Subscriptions to Planet data with Features. This demo will combine tools from three of Planet's APIs with both the Planet Python SDK and the `requests` module to create a workflow that will upload Features in a Collection on the Planet platform, provision quota for those features, then finally create a subscription using your reserved quota.

## Requirements

An account on the [Planet Platform](https://www.planet.com/account/) is required to access any of Planet's APIs. If you are not logged in, you will be prompted to do so below in the *SDK Authentication* section.

It is recommended that you at a minimum, read through the following API Guides before following this workflow. While this workflow is not challenging, the workflow will not go into each step with as many details as the individual API guides.

## Relevant API Guides:

* [Planet SDK Features Demo](https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/api_guides/quota_api/planet_sdk_features_demo.ipynb)
* [Quota Reservation API Introduction](https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/api_guides/quota_api/quota_reservation_api_introduction.ipynb)
* [Subscriptions API Quickstart](https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/api_guides/subscriptions_api/subscriptions_api_quickstart.ipynb)

## Other Useful links 
* [Planet SDK for Python](https://planet-sdk-for-python.readthedocs.io/en/stable/get-started/quick-start-guide/)
* [Planet Python Client Repo](https://github.com/planetlabs/planet-client-python)
* [Planet Features Documentation](https://docs.planet.com/develop/apis/features/)
* [Planet Quota Documentation](https://docs.planet.com/develop/apis/quota/)
* [Planet Subscriptions Documentation](https://docs.planet.com/develop/apis/subscriptions/)

Quota Subscription from a Feature Workflow Steps:
1. Define input parameters for the workflow - AOI, TOI, Products, etc.
2. Create a Feature Collection with the Features API, Upload a Feature to our Feature Collection
3. Submit a Quota Reservation using your new Features with the Quota API
4. Create a Subscription with the Subscriptions API on your Feature

____

## Set up

In order to interact with the Planet APIs with the Python client and `requests`, we need to import the necessary packages and authenticate with our Planet account credentials.

### Imports

In [1]:
import planet
import json
import os
import requests
from datetime import datetime

from planet import Auth, Session, Planet, subscription_request
from planet.geojson import as_ref

QUOTA_URL = "https://api.planet.com/account/v1/quota-reservations/"
ESTIMATE_URL = QUOTA_URL + 'estimate'
PRODUCTS_URL = "https://api.planet.com/account/v1/my/products/"

### SDK Authentication

Normally, your Planet login is used to authenticate and activate the Python SDK. However, since we are using both the SDK and the requests module, we will be authenticating both the SDK and our requests session with our API key.

In [7]:
# Authenticate with the Planet SDK for Python using your API key; See docs: https://docs.planet.com/develop/authentication
# Check for PL_API_KEY environment variable, otherwise type in your API key.
pl_api_key = os.getenv('PL_API_KEY')
if not pl_api_key:
    import getpass
    pl_api_key = getpass.getpass('Planet API Key: ')
    os.environ['PL_API_KEY'] = pl_api_key

auth = planet.Auth.from_key(key=pl_api_key)
pl_session = planet.Session(auth)
pl = planet.Planet(pl_session)

session = requests.Session()
session.auth = (pl_api_key, '')

_____

### Step 1: Define Input Parameters

There will be many variables used in this workflow, and it is good practice to define as many as we can upfront. Here is a list of parameters we will define at the start:

* AOI - Geometry of the area of interest we will use for our feature. We could read this from a geojson path, or define the polygon as a FeatureCollection, which we will do so here with a simple polygon of Briones Reservoir in California.

* TOI - This will take two variables, a `start_time` and an `end_time`. This will be UTC time stamps, and for the demo, will be set to the start and end of December 2025.

* Subscription name - Each subscription must have a unique `name`.

* Delivery Location - There are a few options for this, but all subscriptions are required to be delivered to a cloud bucket or a data hosting service like Sentinel Hub. For the purposes of the Demo, we will be providing code to send the subscription to a Google Cloud Platform storage bucket. 

* Product - You should know the product you will be provisioning your Quota for. For the demo, we will be using ARPS, but you may also follow this workflow for provisioning most Planetary Variables as well. The `product_ID` and `resource_id` will both be retrieved from the Quota API, and used to make the Quota Reservation and Subscription, respectively.


As we run through the workflow, there will be some additional variables we create:

* Feature Reference ID - We will use the Feature Reference IDs returned from when we add our Feature with the Features API to actually reserve our Quota.

In [3]:
### Setup AOI Geometry

briones_geom = [[[-122.22270964735938, 37.90779253018425],[-122.14815875574034,37.90779253018425],[-122.14815875574034, 37.94806920204489],[-122.22270964735938,37.94806920204489],[-122.22270964735938,37.90779253018425]]]

briones_fc = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "id": 1,
      "properties": {
          "title": "Briones Reservoir Area",
          "name_str": "Briones Reservoir",
          "description": "Simple Polygon of the Area around Briones Reservoir in Contra Costa County, California"
      },
      "geometry": {
        "coordinates": briones_geom,
        "type": "Polygon"
      }
    }
  ]
}

In [ ]:
# Alternative: Read your geojson path, then send it to a geo_dict.
geojson_path = 'PATH TO YOUR GEOJSON FILE'

with open(geojson_path, 'r') as file:
    local_fc = json.load(file)

In [4]:
### start_time and end_time for our subscription
start_time = datetime.fromisoformat("2025-12-01T00:00:00Z")
end_time = datetime.fromisoformat("2025-12-31T00:00:00Z")

In [5]:
# Subscription name for when we create the subscription
subscription_name = "briones_arps_subscription"

In [ ]:
# Cloud Storage Destination for subscription delivery

# Google Cloud Storage placeholder
gcs_key = 'gcs_private_key'
bucket_name = 'bucket_name'

bucket_delivery = subscription_request.google_cloud_storage(credentials=gcs_key, bucket=bucket_name)

To set the `product_id` variable for our quota reservation for step 3 and the `resource_id` for our subscription in step 4, we will first check for the products we have Quota available for:

In [23]:
products_response = session.get(PRODUCTS_URL)
products_response.raise_for_status()
product_response_json = products_response.json()

quota_product = product_response_json['results'][0]

In [22]:
product_id = quota_product['id']
resource_id = quota_product['resource_ids'][0]

print(f'Product_id: {product_id}, product resource_id: {resource_id}')

Product_id: 290378, product resource_id: PS_ARD_SR_DAILY


Note that the product ID will be different for each user's quota reservation.

___

### Step 2: Create a Feature Collection and upload Features with the Features API

We will use the Features API to first create a Feature Collection on the Planet Platform, and then upload Features to it to use for our Quota reservation. 

The demo data will use one polygon of the FeatureCollection created above.

In [ ]:
collection_title = "Briones Reservoir"
#set an optional description for your collection
description = "The immediate area around Briones Reservoir"

new_collection_id = pl.features.create_collection(title=collection_title, description=description)
new_collection_id

'briones-reservoir-lvDxKRv'

In [ ]:
feature_ref_ids = pl.features.add_items(collection_id=new_collection_id, feature=briones_fc)

We have set the `feature_ref_ids` variable to the output of `pl.features.add_items`, which is the Feature Reference ID of the Feature we just created. This will be used in the next step to create our Quota Reservation.

Whether or not you're following the demo workflow with the example AOI, `new_items` variable can be plugged directly into your quota reservation, as the `aoi_refs` input for a quota reservation must be input as a list.

____

### Step 3: Submit Quota Reservation

Here we will again use the `requests` module to submit a quota reservation using the Feature Reference ID(s) we created above.

### Optional: Quota Reservation Estimate

There is some space here to optionally compute an estimate of what this quota reservation will cost to your quota. We will print out the remaining quota you will have after you reserve this quota.

In [ ]:
# By default our reservation will display the output in Square Meters.
# If you are going to reserve a large AOI, you can change your units to sqkm, square kilometers.
quota_units = "sqkm"

estimate_payload = {
    "aoi_refs": feature_ref_ids,
    "product_id": product_id,
    "quota_units": quota_units
}

estimate_response = session.post(ESTIMATE_URL, json=estimate_payload)
estimate_response.raise_for_status()
estimate_res_json = estimate_response.json()

pre_reservation = estimate_res_json['quota_remaining']
quota_cost = estimate_res_json['total_cost']

quota_after_cost = pre_reservation - quota_cost
print(f'Your quota remaining after this reservation will be {quota_after_cost} {quota_units}')

Your quota remaining after this reservation will be 99891.39700999999


### Not Optional: Making the Quota Reservation

In [21]:
quota_reservation_payload = {
    "aoi_refs": feature_ref_ids,
    "product_id": product_id,
}

reservation_response = session.post(QUOTA_URL, json=quota_reservation_payload)
print("Status code:", reservation_response.status_code)
reservation_response_json = reservation_response.json()

Status code: 201


In [22]:
reservation_response_json

{'quota_remaining': '99920698505.0',
 'quota_reservations': [{'aoi_ref': 'pl:features/my/briones-reservoir-lvDxKRv/1-07pwdDp',
   'id': 1413942,
   'state': 'reserved'}],
 'quota_total': 100000000000.0,
 'quota_units': 'sqm',
 'quota_used': 79301495.0,
 'url': 'https://api.planet.com/auth/v1/experimental/public/quota-reservations/'}

_____

### Create your Subscription

Now that we have our Features uploaded and our Quota Reserved, we are ready to submit our subscription request with the SDK. 

If you are submitting only one subscription for only one Feature, you can use the variable below, `geom_ref_id` to select it. 

There is a code block at the bottom of this section that will allow you to iterate through the feature_ref_ids variable, creating a subscription for each Feature Reference. Use that code block to create subscriptions with the same Parameters for multiple Features.

In [ ]:
feat_ref_id = feature_ref_ids[0]

In [ ]:

subscription_source = subscription_request.subscription_source(
    source_id=resource_id, geometry=feat_ref_id, start_time=start_time, end_time=end_time)

In [ ]:
sub_built = subscription_request.build_request(
    name=subscription_name, source=subscription_source, delivery=bucket_delivery)

In [ ]:
sub_built

{'name': 'briones_arps_subscription',
 'source': {'parameters': {'id': 'PS_ARD_SR_DAILY',
   'geometry': {'type': 'ref',
    'content': 'pl:features/my/briones-reservoir-lvDxKRv/1-07pwdDp'},
   'start_time': '2025-12-01T00:00:00+00:00',
   'end_time': '2025-12-31T00:00:00+00:00'}},
 'delivery': {'type': 'google_cloud_storage',
  'parameters': {'bucket': '<redacted_for_demo>',
   'credentials': '<redacted_for_demo>'}}}

##### Finally, submit your subscription!!!

In [40]:
sub_created = pl.subscriptions.create_subscription(sub_built)

In [41]:
sub_created

{'name': 'briones_arps_subscription',
 'source': {'type': 'analysis_ready_ps',
  'parameters': {'id': 'PS_ARD_SR_DAILY',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-122.22270964735938, 37.90779253018425],
      [-122.14815875574034, 37.90779253018425],
      [-122.14815875574034, 37.94806920204489],
      [-122.22270964735938, 37.94806920204489],
      [-122.22270964735938, 37.90779253018425]]]},
   'geom_ref': 'pl:features/my/briones-reservoir-lvDxKRv/1-07pwdDp',
   'start_time': '2025-12-01T00:00:00Z',
   'end_time': '2025-12-31T00:00:00Z'}},
 'delivery': {'type': 'google_cloud_storage',
  'parameters': {'bucket': 'simon-test-bucket-pl',
   'credentials': '<REDACTED>'}},
 'created': '2026-01-31T00:29:27.223832Z',
 '_links': {'_self': 'https://api.planet.com/subscriptions/v1/7b2ccb0a-4f79-4cb4-adf8-83467c9c874f',
  'results': 'https://api.planet.com/subscriptions/v1/7b2ccb0a-4f79-4cb4-adf8-83467c9c874f/results'},
 'status': 'preparing',
 'id': '7b2ccb0a-4f79-4cb4-adf8-83

In [ ]:
# Optional multi feature subscriptions creation
all_subscriptions = []

for feat_ref_id in feature_ref_ids:
    subscription_source = subscription_request.subscription_source(
        source_id=resource_id, geometry=feat_ref_id, start_time=start_time, end_time=end_time)

    sub_built = subscription_request.build_request(
        name=subscription_name, source=subscription_source, delivery=bucket_delivery)

    sub_created = pl.subscriptions.create_subscription(sub_built)
    all_subscriptions.append(sub_created)

In [ ]:
all_subscriptions

___

### Workflow Completion

Congratulations, you have completed the Quota Subscription from a Feature Workflow! You should now feel empowered to create other ARPS or Planetary Variables Subscriptions to be delivered to your cloud hosting platforms.